# 该文件未完成

In [ ]:
import openmeteo_requests
import requests_cache
from IPython.core.completerlib import cd_completer
from pandas import DataFrame
from retry_requests import retry
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


def request_openmeteo_data(start_date, end_date, latitude, longitude, daily):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "start_date": start_date,
        "end_date": end_date,
        "latitude": latitude,
        "longitude": longitude,
        "daily": daily
    }
    responses = openmeteo.weather_api(url, params=params)
    return responses

def process_sample_data(responses):
    df_all_sample = pd.DataFrame()
    for response in responses:
        daily = response.Daily()
        daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
        daily_soil_moisture_28_to_100cm_mean = daily.Variables(2).ValuesAsNumpy()
        # 日期
        daily_data = {"date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )}
        # 指标
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["soil_moisture_28_to_100cm_mean"] = daily_soil_moisture_28_to_100cm_mean
        # 整合
        daily_dataframe = pd.DataFrame(data = daily_data)
        df_all_sample = pd.concat([df_all_sample, daily_dataframe], ignore_index=True)
    # 样本合并
    df_l2 = df_all_sample.groupby('date', as_index=False)[['temperature_2m_mean', 'precipitation_sum', 'soil_moisture_28_to_100cm_mean']].mean()


    return df_l2

In [3]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [0.189201],
    "longitude": [116.900165],
    "start_date": "2025-01-01",
    "end_date": "2025-08-05",
    "daily": ["precipitation_sum", "temperature_2m_mean"]
}
responses = openmeteo.weather_api(url, params=params)



# Process 3 locations
for response in responses:
    print(f"\nCoordinates: {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation: {response.Elevation()} m asl")
    print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
    # daily_soil_moisture_7_to_28cm_mean = daily.Variables(2).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}
    print(daily.Interval())

    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    # daily_data["soil_moisture_7_to_28cm_mean"] = daily_soil_moisture_7_to_28cm_mean

    daily_dataframe = pd.DataFrame(data = daily_data)
    print("\nDaily data\n", daily_dataframe)


Coordinates: 0.17574691772460938°N 116.8876724243164°E
Elevation: 13.0 m asl
Timezone difference to GMT+0: 0s
86400

Daily data
                          date  precipitation_sum  temperature_2m_mean
0   2025-01-01 00:00:00+00:00          21.000000            26.458586
1   2025-01-02 00:00:00+00:00           2.400000            25.989836
2   2025-01-03 00:00:00+00:00          16.800001            26.364832
3   2025-01-04 00:00:00+00:00           3.600000            25.637749
4   2025-01-05 00:00:00+00:00          11.200001            26.091913
..                        ...                ...                  ...
212 2025-08-01 00:00:00+00:00           1.000000            28.023169
213 2025-08-02 00:00:00+00:00           3.600000            26.348166
214 2025-08-03 00:00:00+00:00          13.300000            26.173166
215 2025-08-04 00:00:00+00:00          16.500000            25.162750
216 2025-08-05 00:00:00+00:00           1.600000            26.235666

[217 rows x 3 columns]


## open-meteo-api 通用方法

In [13]:
def request_openmeteo_api(latitude:list[str],
                          longitude:list[str],
                          start_date:str,
                          end_date:str,
                          daily_indicators:list[str]) -> pd.DataFrame:
    '''
    The len(latitude) and len(longitude) entered must be equal.
    Latitude would be positive (negative) when a location is in the north (south) of the equator.
    Longitude would be positive (negative) when a location is in the east (west) of the prime meridian.

    :param latitude:    [float, ...]
    :param longitude:   [float, ...]
    :param start_date:  "YYYY-MM-DD"
    :param end_date:    "YYYY-MM-DD"
    :param daily:       [String, ...]
    :return:pd.DataFrame
    '''
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_indicators
    }
    responses = openmeteo.weather_api(url, params=params)

    request_pd = pd.DataFrame()

    for response in responses:
        # 日度指标数据
        daily_datas = response.Daily()
        # 日期列
        daily_dic = {"date": pd.date_range(
            start = pd.to_datetime(daily_datas.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily_datas.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily_datas.Interval()),
            inclusive = "left"
        )}
        # 维度列、经度列
        daily_dic["latitude"] = response.Latitude()
        daily_dic["longitude"] = response.Longitude()

        # 指标列
        i = 0
        for daily_indicator in daily_indicators:
            daily_dic[daily_indicator] = daily_datas.Variables(i).ValuesAsNumpy()
            i = i + 1

        daily_pd = pd.DataFrame(data = daily_dic)
        request_pd = pd.concat([request_pd, daily_pd])

    return request_pd

In [14]:


params = (
    [0.189201, 4.90561],
    [116.900165, 100.695199],
    "2025-01-01",
    "2025-08-05",
    ["precipitation_sum", "temperature_2m_mean", "sunshine_duration"]
)

test_pd = request_openmeteo_api(*params)

